## A Basic of Prompt Engineering

일반적으로 고려해야 할 변수(사용되는 모델의 유형, 애플리케이션의 목표, 지원 인프라)가 너무 많기 때문에 완벽한 프롬프트를 얻기 위한 정해진 규칙은 없다. 그럼에도 불구하고 프롬프트에 적용할 경우 긍정적인 효과를 가져오는 것으로 입증된 몇 가지 명확한 원칙이 있다. 그 중 몇 가지를 살펴보도록 한다.

In [8]:
import numpy as np
import pandas as pd
import openai
from openai import OpenAI
import os

with open('../../config/api.key') as file :
    lines = file.readlines()
    api_key = lines[0].strip()

openai.api_key = api_key # 라이브러리 변수로 지정해야 한다

### 1. 명확한 지침의 제공

모델에 충분한 정보와 지침을 제공함으로써 과업을 정확하고 효율적으로 수행할 수 있도록 한다. 명확한 지침은 다음 요소를 포함한다.

- 과업의 목표나 목적 (예시 : 기사 요약하기, 사실과 의견 구분하기)
- 예상 출력의 형식 또는 구조 (예시 : 각각 10단어 이하로 채워진 글머리 기호 사용)
- 과업의 제약 또는 제한 (예시 : 소스에서 텍스트 복사 금지)
- 과업의 맥락 또는 배경 (예시 : 저널 기사)

예를 들어 모델이 텍스트에서 모든 종류의 지침을 가져와 글머리 기호 목록으로 튜토리얼을 반환한다고 가정해 보자. 또한, 제공된 텍스트에 지침이 없는 경우 모델이 이에 대해 알려줘야 한다.

In [9]:
system_message = """
당신은 주어진 텍스트를 바탕으로 튜토리얼을 생성하는 AI 어시스턴트입니다.
텍스트에 어떤 절차를 진행하는 방법에 대한 지침이 포함되어 있다면,
글머리 기호 목록 형식으로 튜토리얼을 생성하십시오.
그렇지 않으면 텍스트에 지침이 포함되어 있지 않음을 사용자에게 알리십시오.

텍스트 :
"""

instructions = """
이탈리아 제노바에서 유명한 소스를 준비하려면, 먼저 잣을 구워
바질과 마늘과 함께 부엌 절구에 넣어 굵게 다집니다.
그런 다음 절구에 오일의 절반을 넣고 소금과 후추로 간을 합니다.
마지막으로 페스토를 그릇에 옮기고 파르메산 치즈 간 것을 넣고 저어줍니다.
"""

In [10]:
client = OpenAI(api_key = openai.api_key)

response = client.chat.completions.create(
    model = 'gpt-4o-mini',
    messages = [
        {'role': 'system', 'content': system_message},
        {'role': 'user', 'content': instructions},
    ]
)

In [11]:
print(response.choices[0].message.content)

소스를 준비하는 튜토리얼:

- 잣을 구워 준비합니다.
- 구운 잣을 부엌 절구에 넣습니다.
- 바질과 마늘을 추가하고 굵게 다집니다.
- 절구에 오일의 절반을 넣습니다.
- 소금과 후추로 간을 맞춥니다.
- 페스토를 그릇에 옮깁니다.
- 파르메산 치즈를 간 후 넣고 저어줍니다.


모델에 무엇을 시키고자 하는지, 어떻게 수행할지에 관한 명확한 지침을 모델에 제공하여 모델이 이를 이해하도록 도울 수 있다. 이렇게 하면 모델 결과물의 품질과 관련성을 개선하고 추가 수정의 필요성을 줄일 수 있다. 그러나 때로는 명확성만으로는 충분하지 않은 시나리오가 있다. 과업에 관한 관련성을 견고히 하기 위해 우리가 LLM의 사고 방식을 유수할 수도 있다.

### 2. 복잡한 과업을 하위 과업으로 분할

때로 과업이 너무 복잡하거나 모호하여 단일 프롬프트로 처리하기 어려울 수도 있다. 이럴 때에는 여러 프롬프트로 해결할 수 있는 간단한 하위 과업들로 분할하는 것이 좋다. 다음은 복잡한 과업을 하위 과업으로 분할하는 몇 가지 예이다.

1. 텍스트 요약 : 긴 텍스트에 대해 간결하고 정확한 요약을 생성하는 복잡한 과업이다. 이 과업은 다음과 같은 하위 과업들로 분할할 수 있다.
    - 텍스트에서 요점 또는 키워드 추출
    - 요점 또는 키워드를 일관성 있고 유창하게 다시 작성
    - 원하는 길이 또는 형식에 맞게 요약 다듬기
2. 기계 번역 : 한 언어에서 다른 언어로 텍스트를 번역하는 복잡한 과업이다. 이 과업은 다음과 같은 하위 과업으로 나눌 수 있다.
    - 텍스트의 소스 언어 감지
    - 텍스트를 원본 텍스트의 의미와 구조를 보존하는 중간 표현으로 변환
    - 중간 표현에서 목표 언어로 텍스트 생성
3. 시 생성 : 특정 스타일, 주제 또는 분위기를 따르는 시를 만드는 창조적인 과업이다. 이 과업은 다음과 같은 하위 과업으로 나눌 수 있다.
    - 시적 형식(4언절구, 하이쿠, 향가 등)과 운율 체계(ABAB, AABB, ABCB등) 선택
    - 사용자의 입력 또는 선호도에 따라 시의 제목과 주제 생성
    - 선택한 형식, 운율 체계 및 주제와 일치하는 시의 행 또는 구절 생성
    - 일관성, 유창성, 독창성을 보장하기 위해 시를 교정하고 다듬기
4. 코드 생성 : 특정 기능이나 과업을 수행하는 코드를 생성하는 기술적인 과업이다. 이 과업은 다음과 같은 하위 과업으로 나눌 수 있다.
    - 코드에 사용할 프로그래밍 언어(example : Python, C++, JAVA 등)와 프레임워크 또는 라이브러리(example : Tensorflow, Torch, React등) 선택
    - 사용자의 입력 또는 사양에 따라 코드의 함수 이름과 매개변수 및 반환값 목록 생성
    - 코드의 로직과 기능을 구현하는 함수 본문 생성
    - 코드와 사용법을 설명하는 주석 및 문서 추가


모델에 기사의 요약을 생성하도록 요청하는 다음 예제를 살펴 보자. `system_message`에는 과업을 요청하고, article에는 인터넷으로부터 가져온 기사이다.

In [12]:
system_message = """
당신은 기사를 요약하여 사실과 의견, 투자 포인트를 요약하는 AI 어시스턴트입니다.
이 작업을 완료하려면 다음 하위 작업을 수행하십시오 :

언어가 한국어가 아닌 경우 한국어로 우선 번역합니다.
제공된 기사 내용을 종합적으로 읽고 주요 주제와 핵심요점을 식별합니다.
현재 기사 내용을 요약하여 실제로 발생한 사실과 작성자가 제시한 의견을 구분하고 이를 전달하는 단략 요약을 생성합니다.
과정의 각 단계를 출력합니다. 단, 번역 결과는 출력하지 않습니다.
기사 :
"""

article = """
5 big analyst AI moves: Google, Meta stocks exposed to tariff risks; AMD downgrade

Citi trims Nvidia stock price target ahead of earnings
Citi lowered its price target for NVIDIA Corporation (NASDAQ:NVDA) from $175 to $163 ahead of the company’s February 26 earnings report but maintained a Buy rating.
The firm cited concerns over mid-term AI diffusion restrictions and potential weakness in AI infrastructure spending as factors behind the stock’s recent decline.
“We model in-line Jan/Apr-Q and believe expectations are for ~$38B/$42.5B sales for the Jan/Apr-Qs given supply chain indications for the Blackwell to ramp more meaningfully mid this year,” Citi wrote.
Investors are closely watching Nvidia’s earnings and its March GTC conference for updates on the Blackwell architecture and AI spending trends. Nvidia shares have remained range-bound since June, with declining gross margins weighing on sentiment.
“We expect the gross margins to trough in the Apr-Q,” Citi said, noting that China-related risks contributed to a 7% cut in its EPS estimates for the calendar years 2025 and 2026.
Still, Citi remains positive on Nvidia’s long-term potential. “We believe NVDA stock is approaching valuation support levels, trading at mid 20’s P/E & 25% discount to ASIC peer average,” the analysts wrote.
AI demand continues to drive optimism, but near-term uncertainties could keep the stock under pressure until clearer guidance emerges.
Piper Sandler says buy post-earnings weakness in Google stock
Google stock fell more than 7% Wednesday, a day after the company reported mixed Q4 results, but {{0|Piper Sandler}} sees the pullback as a buying opportunity.
3rd party Ad. Not an offer or recommendation by Investing.com. See disclosure here or .
Google owner Alphabet’s (NASDAQ:GOOGL) revenue grew 12% year-over-year in the quarter, slightly exceeding expectations, though EBITDA came in below forecasts.
Search and YouTube performed well, with Search revenue 1% above {{0|Piper Sandler}}’s estimate and YouTube 4% higher, aided by political ad spending. However, Google Cloud, despite growing 30% year-over-year, fell 3% short of the firm’s estimate.
The company’s CEO Sundar Pichai said its AI chatbot has been implemented across all seven of Google’s products, and the highlighted increasing adoption of its Vertex (NASDAQ:VRTX) AI and Flash 2.0.
Meanwhile, the company surpassed its $100 billion Cloud/YouTube revenue run rate target, reaching $110 billion.
“Search growth was led by financial services & retail,” analysts Thomas Champion and James Callahan wrote.
For 2025, new CFO Ruth Porat projected capital expenditures of about $75 billion, well above the Street’s $58 billion estimate and exceeding {{0|Piper Sandler}}’s prior $62 billion forecast. Depreciation growth is also expected to accelerate after rising 28% in 2024.
{{0|Piper Sandler}} slightly trimmed its estimates for Google but remains positive on its fundamentals despite some Cloud capacity constraints.
The analysts maintained an Overweight rating on the stock, saying they are “buyers on weakness.”
Meta, Google stocks ‘most unfavorably positioned’ amid tariff threats, says Wells Fargo (NYSE:WFC)
Wells Fargo analysts see Meta Platforms Inc (NASDAQ:META) and Google as the “most unfavorably positioned” internet stocks following President Trump’s executive orders imposing 25% tariffs on Mexican and most Canadian imports and 10% on goods from China.
The orders also eliminate the Section 321 customs de minimis exemption, which previously allowed imports under $800 to enter the US tariff-free. Analysts noted that while Temu, Shein, and other cross-border merchants had already started near-shoring goods in anticipation of de minimis changes, tariffs on Mexico and Canada now raise costs even for those shipments.
3rd party Ad. Not an offer or recommendation by Investing.com. See disclosure here or .
As a result, "higher prices to American consumers for low-cost cross-border goods" are expected.
Amazon (NASDAQ:AMZN) and Etsy (NASDAQ:ETSY) are viewed as the biggest beneficiaries due to reduced competition from Chinese cross-border sellers. "Closing Section 321 narrows pricing gap; [we] believe ETSY and AMZN are most favorably positioned," the note states.
Analysts estimate that Temu, Shein, and TikTok Shop will generate $78 billion in US e-commerce sales in 2025, up 40% year-over-year, capturing 6% of the market.
For Etsy, lower competition could improve marketing efficiency. A return to customer acquisition costs seen in Q3 2022 could add ~20% to 2025 EBITDA. Amazon is expected to benefit from higher average order values, which analysts estimate will be ~40bps accretive to 2025 operating income margins.
Meanwhile, Meta and Google are "most exposed to cross-border e-commerce trends," with an estimated 11% of Meta’s ad revenue and 7% of Google’s search revenue coming from China-based advertisers.
While Meta's exposure may be somewhat limited by shifting ad spend, Wells Fargo sees a significant impact from tariff-driven changes in cross-border commerce.
Palantir is ‘meme mega-cap’ stock, says ex-JPMorgan top strategist
Former JPMorgan top strategist Marko Kolanovic dubbed Palantir Technologies Inc (NASDAQ:PLTR) a “meme mega-cap” stock after its recent rapid ascent.
In a tweet, Kolanovic questioned its valuation, writing, “No matter how one looks at valuations, doesn't make lot of sense. Congrats to US exceptionalism, now there is a meme Mega cap (+bonus creepy earnings calls).”
Shares jumped 24% on Tuesday, followed by another 9.8% jump on Thursday. The rally has pushed Palantir’s market capitalization past $250 billion, placing it among the largest US public companies.
3rd party Ad. Not an offer or recommendation by Investing.com. See disclosure here or .
Kolanovic’s remark highlights the stock’s rapid rise, driven by excitement over AI and government contracts. Once a niche defense and analytics firm, Palantir has seen its valuation surge on speculation about its AI prospects.
Despite skepticism, bullish investors remain committed, with the stock at the center of a debate between those who see it as an AI leader and others, like Kolanovic, who view its gains as overdone.
AMD stock downgraded amid weakening AI momentum
Advanced Micro Devices Inc (NASDAQ:AMD) tumbled Wednesday after Citi downgraded the stock from Buy to Neutral, citing slowing AI growth, weaker margin leverage, and concerns over inventory buildup. The downgrade extends AMD’s recent downtrend.
Citi also slashed its price target from $175 to $110, noting that AMD’s AI revenue is expected to be flat to down in the first half of 2025, lagging behind peers.
“AMD didn’t provide AI revenue guidance, and it appears AMD’s AI revenue is flat to down for 1H25 with margin dilution,” Citi wrote.
While AMD reported better-than-expected Q4 revenue of $7.66 billion, surpassing consensus ($7.51 billion) and Citi’s estimate ($7.50 billion), analysts raised concerns about a potential CPU inventory build following strong Q4 shipments from both AMD and Intel (NASDAQ:INTC).
Despite 12% sequential revenue growth in Q4, analysts pointed out that gross margin only rose 90 basis points, even as sales climbed 31% in the second half of 2024, signaling weak operating leverage.
AMD’s Q1 guidance of $7.10 billion was slightly above expectations, but Citi warned that softening AI momentum and risks of a CPU inventory correction could pressure future results. The firm cut its 2025 revenue and EPS estimates.
3rd party Ad. Not an offer or recommendation by Investing.com. See disclosure here or .
“We downgrade AMD from Buy to Neutral given slowing AI growth (lower multiple), poor leverage, and risk of an inventory correction,” Citi concluded.
"""

출력을 확인하려면 다음과 같이 실행한다

In [13]:
response = client.chat.completions.create(
    model = 'gpt-4o-mini', # 'gpt-3.5-turbo', 'gpt-4o'
    messages = [
        {'role': 'system', 'content': system_message},
        {'role': 'user', 'content': article},
    ]
)

print(response.choices[0].message.content)

1. **번역되지 않은 원문 확인**: 해당 기사 내용은 영어로 작성되어 있으나, 요약 및 분석은 한국어로 진행됩니다. 

2. **주요 주제 및 핵심 요점 식별**:
   - **NVIDIA**: Citi가 NVIDIA의 주가 목표를 $175에서 $163으로 하향 조정하며, AI 인프라 지출 약세와 중기 AI 확산 제한 우려를 언급. 그럼에도 불구하고 장기 전망은 긍정적으로 평가됨.
   - **구글**: 4분기 실적 발표 후 구글 주가 하락, Piper Sandler는 이를 매수 기회로 판단. AI 챗봇의 모든 제품 적용 등 긍정적 요소 언급.
   - **메타와 구글**: Wells Fargo는 메타와 구글이 관세 위협에 가장 취약한 인터넷 주식으로 지목. 특히 중국 기반 광고 수익의 비율이 높아 관세 변화에 영향을 받을 가능성 있음.
   - **팔란티어**: 전 JPMorgan 전략가가 팔란티어를 ‘밈 메가캡’ 주식으로 언급하며 급등하는 주가에 대한 의문 제기.
   - **AMD**: Citi가 AMD를 매수에서 중립으로 하향 조정하며 AI 성장 둔화 및 재고 누적 우려 언급.

3. **사실과 의견 구분한 요약**:
   - **사실**:
     - NVIDIA의 주가 목표가 하향 조정되었고, AI 인프라 지출 약세가 우려되고 있다.
     - 구글은 4분기 실적이 부진했지만, AI와 관련된 성장이 이루어지고 있다.
     - Wells Fargo는 메타와 구글이 새로운 관세로 인해 가격 상승 영향을 받을 것으로 예상하고 있다.
     - 팔란티어의 주가는 최근 급등하여 시장에서 큰 주목을 받고 있다.
     - AMD는 Citi에 의해 하향 조정되었고, AI 의존도가 낮아지고 있다는 평가를 받았다.
   
   - **의견**:
     - Citi는 NVIDIA의 장기적인 가능성을 긍정적으로 보고 있으나 타격을 받을 수 있다고 언급했다.
     - Piper Sandler는 구글의 주가 하락을 매수 기회로 보았다.
     - 메타와

보다시피, 이 모델은 주어진 기사에서 추출된 주요 주제를 기반으로 고품질의 요약을 생성할 수 있었다. 모델로 하여금 과업을 하위 과업으로 분할하도록 유도했기 떄문에 각 하위 과업의 복잡성을 줄여 최종 결과의 품질을 향상시킬 수 있었다. 이 접근 방식은 모델의 분석적 추론 능력을 향상시키므로 수학적 문제와 같은 시나리오를 다룰 때에도 눈에 띄는 결과를 가져올 수 있다.

복잡한 과업을 더 쉬운 하위 과업으로 분할하는 것은 강력한 기술이긴 하지만, LLM으로 생성된 콘텐츠의 주요 위험 중 하나인 잘못된 출력 문제를 해결하지는 못한다. 또한, 모델에 따라서 동일한 프롬프트에 대해서도 다른 결과를 가져올 수 있다. 예를들어, OpenAI의 GPT에 맞춰진 프롬프트로는 META의 LLAMA에 최적화된 질문이 아닐 수도 있다. 따라서, LLM 모델에 따라서 프롬프트를 다르게 최적화를 진행해야 한다. 

### 3. 정당화 요청

LLM은 이전 토큰들을 기반으로 다음 토큰을 예측하는 방식으로 구축돼 있어 이전에 생성한 내용을 되돌하보지 않는다. 이로 인해 모델이 사용자에게 매우 설득력 있는 방식으로 잘못된 콘텐츠를 출력할 수 있다. LLM기반 애플리케이션이 해당 응답에 대한 구체적인 참조를 제공하지 않는다면 그 이면의 실제적 진실을 검증하기 어려울 수 있다. 따라서, 프롬프트에 몇 가지 참조 및 정당화를 통해 LLM의 답변을 뒷받침하도록 하면 모델이 취한 행동을 되돌리도록 유도할 수 있다. 또한, 정답을 맞혔지만 LLM의 추론 과정이 불명확한 경우에도 정당성을 묻는 것이 유용할 수 있다.

> 답변에 대한 명확한 정당화와 그 뒤의 추론 과정을 제공해 주세요

In [14]:
system_message = """
당신은 추론에 특화된 AI 어시스턴트입니다.
어떠한 사건에 대해서 최선을 다해 원인을 파악해 주세요.
답변에 대한 명확한 정당화와 그 뒤의 추론 과정을 제공해 주세요.
원문이 한국어가 아닌 경우, 한국어로 번역을 하되 번역된 원문은 제공하지 마세요.

제공된 기사 내용을 종합적으로 읽고 주요 주제와 핵심요점을 식별합니다.
현재 기사 내용을 요약하여 실제로 발생한 사실과 작성자가 제시한 의견을 구분하고 이를 전달하는 단략 요약을 생성합니다.
실제로 발생한 사실이 발생하게 된 원인을 기술하세요. 본문에 없다면 일반적인 상식을 얘기해도 무관합니다.
과정의 각 단계를 출력합니다. 단, 번역 결과는 출력하지 않도록 해 주세요.

기사내용 :
"""

text = """
Stock market today: Dow, S&P, Nasdaq slide on Big Tech losses, but weekly gains hold.

US stocks closed the holiday week on a downbeat note as Wall Street slogged to the finish of a largely triumphant year.

The S&P 500 (^GSPC) lost 1.1%, while the tech-heavy Nasdaq Composite (^IXIC) shed 1.5% Friday at the close. The Dow Jones Industrial Average (^DJI) gave up 0.8%. Meanwhile, the 10-year Treasury yield (^TNX) hovered near seven-month highs around 4.6%.

After stacking impressive gains this year, some of the biggest names in tech lost ground as investors took profits, rebalance portfolios, or reassessed their lofty valuations. Tesla (TSLA) lost 5%. Nvidia (NVDA) gave up 2%, while Amazon (AMZN) decreased by 1%.

Wall Street has just three trading days remaining in a 2024 full of big gains, but markets have been unable to mount a "Santa Claus" rally into the end of the year.

Despite the negative day, the S&P 500 logged a weekly gain of 1.8%. The Nasdaq also posted a win of 1.8%, while the Dow notched a 1.5% increase.

Markets have largely digested the year's remaining key economic data points, and investors are now turning their attention to two big themes for the coming year: the Federal Reserve's path for interest rates and the implications of Donald Trump's ascent back to the White House.

On the former, stocks have largely taken in stride the Fed's plans to scale back rate cuts next year after an initial plunge last week. Bets have now shifted squarely to May as the next meeting at which the Fed will slash interest rates, as it continues to grapple with stubborn inflation while keeping a close eye on a cooling labor market.

And on the latter, Yahoo Finance's Ben Werschkul writes that while Trump talked up his big plans during the campaign, especially on the economy, those plans could soon face a reality check from other key power players.
"""

보다시피, 여기서는 meta prompt에서 LLM에 대한 답변을 정당화하고 그 추론도 제공하도록 명시했다. 출력은 아래와 같다.

In [15]:
response = client.chat.completions.create(
    model = 'gpt-4o-mini',
    messages = [
        {'role': 'system', 'content': system_message},
        {'role': 'user', 'content': text}
    ]
)

In [16]:
print(response.choices[0].message.content)

### 주요 주제 및 핵심 요점
- **시장 동향**: 미국 주식 시장이 연말을 맞아 하락세로 마감했으나, 연간 수익은 유지.
- **주요 지수 하락**: S&P 500은 1.1% 하락, Nasdaq은 1.5% 하락, Dow는 0.8% 하락.
- **기술 주식의 하락**: Tesla는 5%, Nvidia는 2%, Amazon은 1% 하락.
- **시장 반응**: 투자자들이 이익 실현을 하면서 주식 가치 재조정 시도.
- **금리 및 정치적 주제**: 금리에 대한 연방준비제도의 계획 및 도널드 트럼프의 복귀가 주요 관전 포인트.

### 발생한 사실과 의견 구분
- **발생한 사실**: 
  - 주식 시장 지수가 하락했다.
  - 특정 대형 기술 주식들이 하락세를 보였다.
  - S&P 500, Nasdaq, Dow 지수는 주간 기준으로는 상승했다.
- **작성자의 의견**: 
  - 투자자들이 수익 실현 및 포트폴리오 재조정을 하고 있다.
  - 금리나 정치적 변화가 투자 환경에 영향을 미칠 것이라는 전망.

### 사건 발생 원인
1. **이익 실현**: 많은 기술 회사들이 올해 중에 큰 상승폭을 기록했기 때문에 투자자들이 이익을 실현하기 위한 매도 활동을 시작했음.
   
2. **포트폴리오 재조정**: 투자자들이 연말을 맞아 자산 구성을 재평가하면서 일부 주식을 매도하고 다른 자산으로 갈아타는 과정에서 주가 하락이 나타났음.

3. **금리 전망**: 연방준비제도의 금리 정책 변화가 다가오고 있으며, 이는 투자자들이 긍정적으로 평가했던 기술주들의 미래가치를 재조정하는 데 영향을 미침.

이런 요인들이 결합되어 주식 시장 전체에 하락세를 유도하며, 특히 기술주들이 부각된 영향을 받았던 것으로 보인다.


정당화는 모델의 출력을 '재고'하게 할 뿐만 아니라 문제를 해결하기 위해 추론이 어떻게 설정되었는지 파악할 수 있게 해주므로 모델을 더욱 안정적이고 견고하게 만드는 훌륭한 도구이다.

비슷한 접근 방식을 사용하면 다양한 프롬프트 수준에서 개입하여 LLM의 성능을 개선할 수도 있다. 예를 들어, 모델이 수학적 문제를 잘못된 방식으로 체계적으로 해결하고 있다는 사실을 발견하면 meta prompt 수준에서 직접 올바른 접근 방식을 제안할 수 있다. 또 다른 예로는 모델에 다양한 추론 기법을 평가하고, meta prompt에서 가장 적합한 것을 프롬프트하기 위해 모델에 여러 가지 결과와 그 정당성을 생성하도록 요청할 수 있다.

### 4. 여러 개의 출력을 생성해 가장 적합한 것을 선택

앞서 살펴본 바와 같이, LLM은 이전 토큰들을 기반으로 다음 토큰을 예측하는 방식으로 구축되며, 이전에 생성한 내용을 되돌아보지 않는다. 이런 경우 만약 하나의 샘플링된 토큰이 부적절하게 선택된다면 LLM은 계속해서 부적절한 토큰을 생성하게 되고, 결과적으로 부적절한 내용을 만들어내게 된다. 주목할 점은 인간과 달리 LLM은 스스로 오류를 인식하고 수정할 수 없다는 것이다. LLM에 물어보면 오류를 인정할 수는 있지만, 그 오류에 대해 생각하고 수정하도록 하려면 명시적으로 프롬프트를 주어야 한다.

이 한계를 극복하는 한 가지 방법은 올바른 토큰을 선택할 가능성을 높이기 위해 더 다양한 선택지를 고려하는 것이다. 하나의 응답만 생성하는 대신 모델에 여러 개의 응답을 생성하도록 유도한 다음 사용자의 질의에 가장 적합한 응답을 선택하도록 할 수 있다. 이렇게 하면 LLM의 작업이 다음 두 가지 하위 과업으로 나뉜다.

1. 사용자의 질의에 대해 여러 개의 응답 생성하기
2. meta prompt에서 지정할 수 있는 몇 가지 기준에 따라 이러한 응답을 비교하고 가장 적합한 응답 선택하기

이전에 살펴본 어시스턴트의 후속 예시를 살펴보도록 하자

In [17]:
system_message = """
당신은 추론에 특화된 AI 어시스턴트입니다.
어떠한 사건에 대해서 최선을 다해 원인을 파악해 주세요.
답변에 대한 명확한 정당화와 그 뒤의 추론 과정을 제공해 주세요.
원문이 한국어가 아닌 경우, 한국어로 번역을 하되 번역된 원문은 제공하지 마세요.

제공된 기사 내용을 종합적으로 읽고 주요 주제와 핵심요점을 식별합니다. 
현재 기사 내용을 요약하여 실제로 발생한 사실과 작성자가 제시한 의견을 구분하고 이를 전달하는 단략 요약을 생성합니다.
주어진 기사에 대해 세 가지 답변을 생성해야 합니다.
각 답변에는 실제로 발생한 사실이 발생하게 된 원인을 기술하세요. 본문에 없다면 일반적인 상식을 얘기해도 무관합니다.
그런 다음, 세 가지 답변 중에서 원인과 결과를 고려했을 때 가장 그럴듯한 것을 선택하세요.

기사내용 :
"""

text = """
Stock market today: Dow, S&P, Nasdaq slide on Big Tech losses, but weekly gains hold.

US stocks closed the holiday week on a downbeat note as Wall Street slogged to the finish of a largely triumphant year.

The S&P 500 (^GSPC) lost 1.1%, while the tech-heavy Nasdaq Composite (^IXIC) shed 1.5% Friday at the close. The Dow Jones Industrial Average (^DJI) gave up 0.8%. Meanwhile, the 10-year Treasury yield (^TNX) hovered near seven-month highs around 4.6%.

After stacking impressive gains this year, some of the biggest names in tech lost ground as investors took profits, rebalance portfolios, or reassessed their lofty valuations. Tesla (TSLA) lost 5%. Nvidia (NVDA) gave up 2%, while Amazon (AMZN) decreased by 1%.

Wall Street has just three trading days remaining in a 2024 full of big gains, but markets have been unable to mount a "Santa Claus" rally into the end of the year.

Despite the negative day, the S&P 500 logged a weekly gain of 1.8%. The Nasdaq also posted a win of 1.8%, while the Dow notched a 1.5% increase.

Markets have largely digested the year's remaining key economic data points, and investors are now turning their attention to two big themes for the coming year: the Federal Reserve's path for interest rates and the implications of Donald Trump's ascent back to the White House.

On the former, stocks have largely taken in stride the Fed's plans to scale back rate cuts next year after an initial plunge last week. Bets have now shifted squarely to May as the next meeting at which the Fed will slash interest rates, as it continues to grapple with stubborn inflation while keeping a close eye on a cooling labor market.

And on the latter, Yahoo Finance's Ben Werschkul writes that while Trump talked up his big plans during the campaign, especially on the economy, those plans could soon face a reality check from other key power players.
"""

이 경우 모델에 원문에 대한 세 가지 답을 생성한 다음, 가장 가능성이 높은 답을 제시하고 그 이유를 설명하도록 요청했다. 결과를 확인해 보도록 하자.

In [18]:
response = client.chat.completions.create(
    model = 'gpt-4o-mini',
    messages = [
        {'role': 'system', 'content': system_message},
        {'role': 'user', 'content': text}
    ]
)

In [19]:
print(response.choices[0].message.content)

1. **주요 사실**: 미국 주식 시장에서 이번 주 금요일, S&P 500이 1.1%, Nasdaq Composite가 1.5%, Dow Jones Industrial Average가 0.8% 하락했습니다. 큰 기술 기업들이 투자자들에게 이익 실현 등의 이유로 하락세를 보였습니다. Tesla는 5%, Nvidia는 2%, Amazon은 1% 하락했습니다. 하지만, 주 및 나스닥은 이번 주에 각각 1.8%의 주간 상승률을 기록했습니다.

   **원인**: 기술 기업들의 하락은 이익 실현과 포트폴리오의 재조정으로 인한 것으로, 시장 참가자들이 해당 기업들의 높은 평가를 재평가하기 시작했기 때문입니다.

2. **주요 사실**: 올해 남은 주요 경제 데이터에 대한 시장의 반응이 있었으며, 투자자들은 내년의 금리에 대한 연방준비제도(Fed)의 방향과 도널드 트럼프의 재임 가능성에 관심을 돌리고 있습니다. 

   **원인**: 연준이 내년 금리 인하를 축소할 계획을 세우고, 노동 시장의 냉각을 주의 깊게 살펴보는 가운데, 투자자들이 금리 인하 시점을 미루고 있기 때문에 주식 시장에도 영향을 미치고 있습니다.

3. **주요 사실**: 연준의 금리 결정과 트럼프의 귀환이 투자자들이 주목하는 두 가지 주요 주제로 부각되고 있습니다. 맥락에서 투자자들은 내년의 경제 계획에 대한 현실 점검을 해야 할 것입니다.

   **원인**: 대선 캠페인에서 트럼프가 내세우는 경제 계획이 실제로 실현될 수 있을지에 대한 불확실성이 투자자들에게 큰 영향을 미치고 있으며, 이는 경제 전반에 걸쳐 기대감을 조성하면서도 동시에 걱정도 불러일으키고 있습니다.

**가장 그럴듯한 답변 선택**: 첫 번째 답변이 가장 그럴듯합니다. 이는 기술 주식의 하락이 실제로 발생한 증거와 관련이 깊으며, 이익 실현이 현재의 시장 상황에서 주요한 요인이라는 점에서 돋보입니다. 이렇게 기술 주식의 높은 평가가 재평가되는 과정은 하락세의 주된 원인이 될 수 있습니다.


보다시피, 이 모델은 가장 그럴듯한 답을 선택하면서 그 선택의 정당성도 함께 제시하였다.

앞서 설명한 것처럼 모델에 다양한 접근 방식으로 문제를 해결하도록 강제하는 것은 여러 추론 샘플을 수집하는 방법이며, 이는 meta prompt에서 추가 지침으로 사용될 수 있다. 예를 들어, 모델이 항상 문제에 대한 가장 간단한 해결책이 아닌 다른 것을 제안하도록 하려면, 즉 '다르게 생각하도록' 하려면, 모델에 N가지 방법으로 문제를 해결하도록 강제한 다음 가장 창의적인 추론을 meta prompt의 framework로 사용할 수 있다.

마지막으로 살펴볼 요소는 meta prompt에 부여할 전체 구조이다. 실제로 이전 예제에서 몇 가지 구문과 명령어가 포함된 시스템 메세지 샘플을 살펴본 바 있다. 앞으로는 이런 구문과 명령어의 순서와 '강도'가 어떻게 불변이 아닌지 살펴보도록 하자.

### 5. 마지막에 지침을 반복

LLM은 meta prompt의 모든 섹션에 동일한 가중치 또는 중요도를 두어 처리하지 않는 경향이 존재한다. micro soft의 software engineer, John Stewart는 블로그 게시물 <Large Language Model Prompt Engineering for Complex Summarization>에서 prompt 섹션을 배열하는 것에 대한 흥미로운 결과를 발견했다. 좀 더 구체적으로, 그는 몇가지 실험을 통해 prompt 끝에 주된 지침을 반복하면 모델이 자체적인 최근성 편향을 극복하는 데 도움이 될 수 있음을 발견하였다.

최근성 편향이란, 프롬프트의 마지막에 나타나는 정보에 더 많은 가중치를 부여하고 그 이전에 나타나는 정보는 무시하거나 잊어버리는 LLM의 경향을 말한다. 이로 인해 과업의 전체 맥락을 고려하지 않은 부정확하거나 일관성 없는 응답이 나올 수 있다. 예를 들어, 프롬프트가 두 사람 간의 긴 대화인 경우 모델은 마지막 몇 개의 메세지에만 집중하고, 이전 메세지는 무시할 수 있다.

**최근성 편향을 극복하는 몇 가지 방법**

- 과업을 더 작은 단계 또는 하위 과업으로 나누고 그 과정에서 피드백이나 안내를 제공한다
- 프롬프트 끝에 지침이나 과업의 주요 목표를 반복한다

In [20]:
system_message = """
당신은 금융, 경제에 종사하고 있는 사람들의 연설을 전문적으로 분석하는 분석가입니다. 
연설의 감정을 향후 주식시장에 미칠 영향을 기준으로 긍정, 부정, 중립으로 분류합니다.
감정만을 구두점 없이 출력합니다.

원문이 한국어가 아닌 경우, 한국어로 번역을 하되 번역된 원문은 제공하지 마세요.

연설 : 
"""

speech = """
Good afternoon. Thank you to the World Affairs Council, the Federal Reserve Bank of Dallas, and the Dallas Regional Chamber for the kind invitation to be with you today. I will start with some brief comments on the economy and monetary policy.

Looking back, the U.S. economy has weathered a global pandemic and its aftermath and is now back to a good place. The economy has made significant progress toward our dual-mandate goals of maximum employment and stable prices. The labor market remains in solid condition. Inflation has eased substantially from its peak, and we believe it is on a sustainable path to our 2 percent goal. We are committed to maintaining our economy's strength by returning inflation to our goal while supporting maximum employment.

Recent Economic Data
Economic growth
The recent performance of our economy has been remarkably good, by far the best of any major economy in the world. Economic output grew by more than 3 percent last year and is expanding at a stout 2.5 percent rate so far this year. Growth in consumer spending has remained strong, supported by increases in disposable income and solid household balance sheets. Business investment in equipment and intangibles has accelerated over the past year. In contrast, activity in the housing sector has been weak.

Improving supply conditions have supported this strong performance of the economy. The labor force has expanded rapidly, and productivity has grown faster over the past five years than its pace in the two decades before the pandemic, increasing the productive capacity of the economy and allowing rapid economic growth without overheating.

The labor market
The labor market remains in solid condition, having cooled off from the significantly overheated conditions of a couple of years ago, and is now by many metrics back to more normal levels that are consistent with our employment mandate. The number of job openings is now just slightly above the number of unemployed Americans seeking work. The rate at which workers quit their jobs is below the pre-pandemic pace, after touching historic highs two years ago. Wages are still increasing, but at a more sustainable pace. Hiring has slowed from earlier in the year. The most recent jobs report for October reflected significant effects from hurricanes and labor strikes, making it difficult to get a clear signal. Finally, at 4.1 percent, the unemployment rate is notably higher than a year ago but has flattened out in recent months and remains historically low.

Inflation
The labor market has cooled to the point where it is no longer a source of significant inflationary pressures. This cooling and the substantial improvement in broader supply conditions have brought inflation down significantly over the past two years from its mid-2022 peak above 7 percent. Progress on inflation has been broad based. Estimates based on the consumer price index and other data released this week indicate that total PCE prices rose 2.3 percent over the 12 months ending in October and that, excluding the volatile food and energy categories, core PCE prices rose 2.8 percent. Core measures of goods and services inflation, excluding housing, fell rapidly over the past two years and have returned to rates closer to those consistent with our goals. We expect that these rates will continue to fluctuate in their recent ranges. We are watching carefully to be sure that they do, however, just as we are closely tracking the gradual decline in housing services inflation, which has yet to fully normalize. Inflation is running much closer to our 2 percent longer-run goal, but it is not there yet. We are committed to finishing the job. With labor market conditions in rough balance and inflation expectations well anchored, I expect inflation to continue to come down toward our 2 percent objective, albeit on a sometimes-bumpy path.

Monetary Policy
Given progress toward our inflation goal and the cooling of labor market conditions, last week my Federal Open Market Committee colleagues and I took another step in reducing the degree of policy restraint by lowering our policy interest rate 1/4 percentage point.

We are confident that with an appropriate recalibration of our policy stance, strength in the economy and the labor market can be maintained, with inflation moving sustainably down to 2 percent. We see the risks to achieving our employment and inflation goals as being roughly in balance, and we are attentive to the risks to both sides. We know that reducing policy restraint too quickly could hinder progress on inflation. At the same time, reducing policy restraint too slowly could unduly weaken economic activity and employment.

We are moving policy over time to a more neutral setting. But the path for getting there is not preset. In considering additional adjustments to the target range for the federal funds rate, we will carefully assess incoming data, the evolving outlook, and the balance of risks. The economy is not sending any signals that we need to be in a hurry to lower rates. The strength we are currently seeing in the economy gives us the ability to approach our decisions carefully. Ultimately, the path of the policy rate will depend on how the incoming data and the economic outlook evolve.

We remain resolute in our commitment to the dual mandate given to us by Congress: maximum employment and price stability. Our aim has been to return inflation to our objective without the kind of painful rise in unemployment that has often accompanied past efforts to bring down high inflation. That would be a highly desirable result for the communities, families, and businesses we serve. While the task is not complete, we have made a good deal of progress toward that outcome.

Thank you, and I look forward to our discussion.
"""

이 시나리오에서는 연설 이전에 지침이 있으므로 모델을 초기화하고 `system_message`와 `speech`이라는 두 변수를 모델에 입력해 보자. 

In [21]:
response = client.chat.completions.create(
    model = 'gpt-4o-mini',
    messages = [
        {'role': 'system', 'content': system_message},
        {'role': 'user', 'content': speech}
    ]
)

프롬프트 끝 부분에 감정만 출력하라는 지침을 두 번 반복하였다. 문맥 창에 계속 저장해야 하는 대화 내역이 있을 떄 창의 앞부분에 주요 지침이 있으면 모델이 전체 기록을 살펴본 후에 이를 염두에 두지 않게 되어 효력이 떨어질 수 있는데, 그럴 때 이와 같이 주요 지침을 끝에서 다시 한 번 반복해 주면 좋다.

In [22]:
print(response.choices[0].message.content)

긍정


### 6. 구분선을 사용

마지막으로 다룰 원칙은 meta prompt의 형식과 관련된 것이다. 이는 LLM의 의도를 더 잘 이해하고 서로 다른 섹션과 단락을 서로 연관시키는데 도움이 된다. 이를 위해 prompt내에 구분선(delimiter)을 넣을 수 있다. 구분선은 개념이 아닌 구조를 명확히 표시하는 연속된 문자 혹은 기호가 될 수 있다. 아래와 같은 예시가 존재한다

- `>>>>`
- `====`
- `-----`
- `####`

구분선의 사용에는 다음과 같은 장점이 있다.

- 명확한 구분 : 구분선은 프롬프트 내에서 별도의 섹션을 표시하여 지침, 예제 및 원하는 출력을 구분한다
- LLM을 위한 지침 : 구분선을 적절히 사용하면 모호성을 제거하여 모델을 효과적으로 안내할 수 있다.
- 정확성 향상 : 구분선은 즉각적인 이해도를 높여 보다 관련성 높은 응답을 제공한다.
- 일관성 향상 : 구분선을 효과적으로 사용하면 지침, 입력 및 출력을 정리하여 일관된 응답을 유도할 수 있다.

아래의 예시로, 모델에 사용자의 과업을 파이썬 코드로 변환하도록 지시하고 이를 위한 예제를 제공하는 meta prompt를 고려해 보자.

In [23]:
system_message = """
당신은 포트폴리오 구성을 하는 매니저의 요청에 따라 코드를 작성하는 파이썬 프로그래머입니다. 요청에 따라 파이썬 코드를 생성합니다.

==> 예제 시작

--사용자 질문--
Monte Carlo Simulation과 Markowitz의 포트폴리오 이론을 이용해 임의의 주식 5개에 대해서 최적의 배분 비율을 도출하는 함수를 작성해 주세요.
input은 pandas.DataFrame형태이며, 5개의 자산이 컬럼으로 들어가 있습니다.
각 자산 데이터는 수익률 데이터입니다.

--사용자 출력--
예시로, 함수는 다음과 같이 작성되어야 합니다.
```
def portfolio_optimization(data : pd.DataFrame) -> pd.DataFrame:
    '''
    this function is responsible for optimizing the portfolio
    '''
    return result
```
<=== 예제 끝
"""

query = "5개의 자산을 임의로 생성하고, 이 자산들의 최적 weight를 구하는 파이썬 함수를 생성하세요."

위의 예제에서는 구분선을 사용해 one shot 학습 방식을 위한 예제의 시작과 끝을 모두 지정하고 예제 내에서 파이썬 코드를 지정하였다.

In [24]:
response = client.chat.completions.create(
    model = 'gpt-4o-mini',
    messages = [
        {'role': 'system', 'content': system_message},
        {'role': 'user', 'content': query}
    ]
)

결과는 다음과 같이 출력된다.

In [25]:
print(response.choices[0].message.content)

아래는 5개의 임의의 자산을 생성하고, Monte Carlo Simulation을 사용하여 Markowitz의 포트폴리오 이론에 따라 최적의 배분 비율을 도출하는 함수를 작성한 예제입니다. 이 함수는 사용자로부터 입력받은 데이터프레임을 기반으로 최적의 포트폴리오 배분 비율을 반환합니다.

```python
import numpy as np
import pandas as pd

def generate_random_asset_data(num_assets=5, num_samples=1000):
    """Generate random asset return data"""
    return pd.DataFrame(np.random.randn(num_samples, num_assets) / 100, columns=[f'Asset_{i+1}' for i in range(num_assets)])

def portfolio_optimization(data: pd.DataFrame, num_portfolios=10000) -> pd.DataFrame:
    """
    This function optimizes the portfolio based on Monte Carlo Simulation and Markowitz theory.

    Parameters:
    data (pd.DataFrame): DataFrame containing the returns of the assets.
    num_portfolios (int): Number of portfolios to simulate.

    Returns:
    pd.DataFrame: DataFrame with optimal weights and expected returns/risk metrics.
    """
    results = np.zeros((3, num_portfolios))
    weights_record = []

    for i in range(num_portfolios):
   

출력된 코드도 입력할 때와 마찬가지로 역따옴표를 포함한 것을 볼 수 있다. 실제로, 프롬프트에서 생성한 코드를 직접 실행해 보면 아래와 같이 성공적으로 실행이 되는 것을 볼 수 있다.

In [26]:
import numpy as np
import pandas as pd

def generate_random_asset_data(num_assets=5, num_samples=1000):
    """Generate random asset return data"""
    return pd.DataFrame(np.random.randn(num_samples, num_assets) / 100, columns=[f'Asset_{i+1}' for i in range(num_assets)])

def portfolio_optimization(data: pd.DataFrame, num_portfolios=10000) -> pd.DataFrame:
    """
    This function optimizes the portfolio based on Monte Carlo Simulation and Markowitz theory.

    Parameters:
    data (pd.DataFrame): DataFrame containing the returns of the assets.
    num_portfolios (int): Number of portfolios to simulate.

    Returns:
    pd.DataFrame: DataFrame with optimal weights and expected returns/risk metrics.
    """
    results = np.zeros((3, num_portfolios))
    weights_record = []

    for i in range(num_portfolios):
        weights = np.random.random(data.shape[1])
        weights /= np.sum(weights)  # Normalize weights to sum to 1
        weights_record.append(weights)
        
        portfolio_return = np.sum(data.mean() * weights) * 252  # Expected annual return
        portfolio_std_dev = np.sqrt(np.dot(weights.T, np.dot(data.cov() * 252, weights)))  # Expected annual risk
        
        results[0, i] = portfolio_return
        results[1, i] = portfolio_std_dev
        results[2, i] = results[0, i] / results[1, i]  # Sharpe ratio, assuming risk-free rate is 0
    
    max_sharpe_idx = np.argmax(results[2])
    optimal_weights = weights_record[max_sharpe_idx]
    result = pd.DataFrame({
        'Asset': data.columns,
        'Weight': optimal_weights
    })
    
    return result

In [27]:
# Generate random asset data
asset_data = generate_random_asset_data()

# Optimize the portfolio
optimal_portfolio = portfolio_optimization(asset_data)
print(optimal_portfolio)

     Asset    Weight
0  Asset_1  0.210891
1  Asset_2  0.298549
2  Asset_3  0.107113
3  Asset_4  0.003588
4  Asset_5  0.379859
